##Vehicle Detection and Tracking in Riyadh

Group : Wafa ALDALBAHI, Arwa Alanzi, Arwa altwerqi , Hawra Aljishi

###Introduction
The provided code represents a  implementation of a vehicle detection and tracking system tailored specifically  Riyadh city. Leveraging the power of deep learning and computer vision, the system aims to address the challenges of traffic congestion and road safety by providing real-time insights into vehicular movement.

**1. Initialization and Setup:**

*  The code begins with the initialization of essential parameters and constants, including model names, video paths, resolution settings, and confidence thresholds.
*  It imports necessary libraries such as NumPy for numerical computations, defaultdict and deque for efficient data management, tqdm for progress tracking, and supervision and Ultralytics for implementing machine learning and computer vision functionalities.

In [1]:
pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.8/778.8 kB 6.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
pip install supervision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 2.5 MB/s eta 0:00:00


In [3]:
pip install moviepy

In [4]:
# Import necessary libraries
import numpy as np
from collections import defaultdict, deque
from tqdm import tqdm
import supervision as sv
from ultralytics import YOLO

**2.  Model Initialization and Configuration:**

It initializes the YOLO model (YOLOv9) from the Ultralytics library for vehicle detection.
Additionally, it sets up the BYTETracker from the supervision library for object tracking and initializes other components such as VideoInfo and frame generators.

In [5]:
# Define constants and parameters
MODEL_NAME = 'yolov9c.pt'  # Replace with the latest  model
SOURCE_VIDEO_PATH = '/content/R1.MP4'  # Replace with your source video path
TARGET_VIDEO_PATH = 'output7.mp4'  # Replace with your target video path
MODEL_RESOLUTION = 640  # Resolution for model inference
CONFIDENCE_THRESHOLD = 0.20  # Adjust this threshold as needed
IOU_THRESHOLD = 0.5
selected_classes = [0, 1, 2]  # Replace with your desired class IDs

In [6]:
LINE_START = sv.Point(50, 1500)
LINE_END = sv.Point(3840-50, 1500)

In [7]:
# Create YOLO model instance
model = YOLO(MODEL_NAME)

100%|██████████| 6.23M/6.23M [00:00<00:00, 270MB/s]


In [8]:
# Create BYTETracker instance
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=30, match_thresh=0.8, frame_rate=30)

SupervisionWarnings: `track_buffer` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'lost_track_buffer' instead.
SupervisionWarnings: `track_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'track_activation_threshold' instead.
SupervisionWarnings: `match_thresh` in `ByteTrack.__init__` is deprecated and will be remove in `supervision-0.23.0`. Use 'minimum_matching_threshold' instead.


In [9]:
# Create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)


In [10]:
# Create frame generator
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

In [11]:
# Create LineZone instance, previously called LineCounter class
line_zone = sv.LineZone(start=LINE_START, end=LINE_END)

In [12]:
# Set default values for thickness and text scale
thickness = 2  # Default line thickness
text_scale = 0.5  # Smaller text scale

**3. Annotation and Visualization Setup:**

The code sets up annotators for visualizing bounding boxes, traces, and line zones on the video frames.
It defines the line zone for monitoring vehicle movement across a specific section of the road.

In [13]:
# Create annotators
box_annotator = sv.BoxAnnotator(thickness=thickness, text_thickness=thickness, text_scale=text_scale)
trace_annotator = sv.TraceAnnotator(thickness=thickness, trace_length=video_info.fps * 2)
line_zone_annotator = sv.LineZoneAnnotator(thickness=thickness, text_thickness=thickness, text_scale=text_scale)

In [14]:
# Define coordinates for speed calculation
coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))

**4. Frame Processing and Callback Function:**

The core functionality lies within the callback function, which processes each frame of the video.
It performs object detection using the YOLO model on the frame and converts the results into supervision Detections.
Object tracking is then applied to the detections using the BYTETracker, and detection coordinates are stored for speed calculation.
Based on the detected vehicles, labels are generated, including class names, confidence scores, and calculated speeds.
The annotated frame is generated with visualizations of traces, bounding boxes, labels, and line zone status.

**5. Video Processing:**

The code iterates through each frame of the video, applying the callback function to process and annotate each frame.
The annotated frames are compiled into a new video with the specified target path.

In [15]:


# Define callback function for video processing
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # Model prediction on a single frame and conversion to supervision Detections
    results = model(frame, imgsz=MODEL_RESOLUTION, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)

    # Only consider class IDs from selected_classes
    detections = detections[np.isin(detections.class_id, selected_classes)]

    # Tracking detections
    detections = byte_tracker.update_with_detections(detections)

    # Store detection coordinates for speed calculation
    points = detections.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)
    for tracker_id, [_, y] in zip(detections.tracker_id, points):
        coordinates[tracker_id].append(y)

    # Generate labels and calculate speed
    labels = []
    num_cars = len(detections)
    for confidence, class_id, tracker_id in zip(detections.confidence, detections.class_id, detections.tracker_id):
        class_name = model.model.names[class_id]
        if len(coordinates[tracker_id]) < video_info.fps / 2:
            labels.append(f"#{tracker_id} {class_name} {confidence:.2f}")
        else:
            coordinate_start = coordinates[tracker_id][-1]
            coordinate_end = coordinates[tracker_id][0]
            distance = abs(coordinate_start - coordinate_end)
            time = len(coordinates[tracker_id]) / video_info.fps
            speed = distance / time * 3.6  # Convert to km/h
            if num_cars >= 50:
                speed = max(speed - 10, 40)
            elif num_cars <= 10:
                speed = min(speed + 10, 120)
            labels.append(f"#{tracker_id} {class_name} {confidence:.2f} {int(speed)} km/h")

    # Annotate frame with traces, bounding boxes, and labels
    annotated_frame = trace_annotator.annotate(scene=frame.copy(), detections=detections)
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

    # Update line zone with current detections
    line_zone.trigger(detections)

    # Annotate frame with line zone results and return
    return line_zone_annotator.annotate(annotated_frame, line_counter=line_zone)

# Process the whole video
sv.process_video(
    source_path=SOURCE_VIDEO_PATH,
    target_path=TARGET_VIDEO_PATH,
    callback=callback
)


SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `BoundingBoxAnnotator` and `LabelAnnotator` instead
SupervisionWarnings: annotate is deprecated: `BoxAnnotator` is deprecated and will be removed in `supervision-0.22.0`. Use `B

**References :**
1.  Speed Estimation & Vehicle Tracking | Computer Vision | Open Source: https://www.youtube.com/watch?v=uWP6UjDeZvY
2.  Track & Count Objects using YOLOv8 ByteTrack & Supervision : https://www.youtube.com/watch?v=OS5qI9YBkfk
